In [96]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import timeit
import re
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
from IPython.display import Image
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import plotly.express as px

In [97]:
election_results = pd.read_csv("../../webscraping_open_secrets-silver-ish-sharks/data/election_results.csv")

In [98]:
election_results.rename(columns={})

election_results

,Unnamed: 0,Candidate_name,Party,State,District,Incumbent,Winner,Raised,Spent,percent_raised,percent_spent
0,0,Jerry Carl,(R),Alabama,1,False,True,1971321,1859349,96.095624,95.925703
1,1,James Averhart,(D),Alabama,1,False,False,80095,78973,3.904376,4.074297
2,0,Barry Moore,(R),Alabama,2,False,True,650807,669368,92.070532,92.281307
3,1,Phyllis Harvey,(D),Alabama,2,False,False,56050,55988,7.929468,7.718693
4,0,Mike D,D,Alabama,3,True,True,1193111,1218564,95.956760,96.747133
...,...,...,...,...,...,...,...,...,...,...,...
889,1,Tricia Zunker,(D),Wisconsin,7,False,False,1261957,1232690,32.362718,32.894277
890,0,Mike Gallagher,(R),Wisconsin,8,True,True,3202905,2841801,88.480898,87.663451
891,1,Amanda Stuck,(D),Wisconsin,8,False,False,416978,399916,11.519102,12.336549
892,0,Liz Cheney,(R),Wyoming,1,True,True,3003883,3060167,95.711395,95.857821


In [99]:
URL_states = "https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120"
URL_abb = "https://www.50states.com/abbreviations.htm"

response_states = requests.get(URL_states)
response_abb = requests.get(URL_abb)

soup = BS(response_states.text)

soup_abb = BS(response_abb.text)

abb = pd.read_html(str(soup_abb.find('table')))[0]
abb = abb.drop(columns='STANDARD ABBREVIATION')
abb = abb.rename(columns={'US STATE':'state', 'POSTAL ABBREVIATION':'state_abb'})


states = pd.read_html(str(soup.find('table')))[0]
states = states.drop([50])

state_districts = pd.merge(states, abb, on='state')
state_districts['representatives'] = state_districts['representatives'].apply(pd.to_numeric)
display(state_districts.head())
district_count = "Number of US Representatives: " + str(state_districts['representatives'].sum())
district_count

,state,representatives,state_abb
0,Alabama,7,AL
1,Alaska,1,AK
2,Arizona,9,AZ
3,Arkansas,4,AR
4,California,53,CA


'Number of US Representatives: 435'

In [100]:
incumbent_data = election_results[(election_results['Incumbent'] == True) & (election_results['Winner'] == True)]
incumbent_data

,Unnamed: 0,Candidate_name,Party,State,District,Incumbent,Winner,Raised,Spent,percent_raised,percent_spent
4,0,Mike D,D,Alabama,3,True,True,1193111,1218564,95.956760,96.747133
6,0,Robert B,B,Alabama,4,True,True,1255076,1323812,96.459423,96.710075
8,0,Mo Brooks,(R),Alabama,5,True,True,655365,210045,100.000000,100.000000
9,0,Gary Palmer,(R),Alabama,6,True,True,907219,909082,100.000000,100.000000
10,0,Terri Sewell,(D),Alabama,7,True,True,2168165,1495957,100.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...
883,0,Gwen Moore,(D),Wisconsin,4,True,True,1183995,1166739,100.000000,100.000000
886,0,Glenn S,S,Wisconsin,6,True,True,1815756,1736659,86.320334,87.601787
888,0,Tom Tiffany,(R),Wisconsin,7,True,True,2637459,2514740,67.637282,67.105723
890,0,Mike Gallagher,(R),Wisconsin,8,True,True,3202905,2841801,88.480898,87.663451


In [107]:
incumbent_agg = incumbent_data['State'].value_counts().reset_index()
incumbent_agg = incumbent_agg.rename(columns={'index':'state', 'State':'numb_incumb'})
incumbent_agg

,state,numb_incumb
0,California,47
1,Texas,29
2,Florida,22
3,New York,21
4,Pennsylvania,17
5,Ohio,16
6,Illinois,16
7,Michigan,12
8,New Jersey,12
9,Virginia,10


In [109]:
incumbent_agg = pd.merge(state_districts, incumbent_agg, how='outer').fillna(value=0)
incumbent_agg

,state,representatives,state_abb,numb_incumb
0,Alabama,7,AL,5.0
1,Alaska,1,AK,1.0
2,Arizona,9,AZ,9.0
3,Arkansas,4,AR,4.0
4,California,53,CA,47.0
5,Colorado,7,CO,6.0
6,Connecticut,5,CT,5.0
7,Delaware,1,DE,1.0
8,Florida,27,FL,22.0
9,Georgia,14,GA,10.0
